In [1]:
import azureml.core 
print(azureml.core.VERSION)

1.20.0


In [ ]:
pip install --upgrade --upgrade-strategy eager azureml-sdk[automl,widgets,notebooks]

In [2]:
import azureml.core 
print(azureml.core.VERSION)

1.20.0


In [3]:
from azureml.core.environment import Environment
Environment(name="vaenv")
myenv = Environment(name="vaenv")

In [4]:
from azureml.core.conda_dependencies import CondaDependencies

dependencies = CondaDependencies()
dependencies.add_pip_package("scikit-learn")
dependencies.add_pip_package("numpy==1.15.4")
myenv.python.conda_dependencies=dependencies

In [5]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name='Toria_workspace',
            subscription_id='784d8740-4c70-4d20-8f45-b2bfecf94029',
            resource_group='myresourcegroup',)
exp = Experiment(workspace=ws, name="Toria_workspace")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: Toria_workspace
Azure region: eastus2
Subscription id: 784d8740-4c70-4d20-8f45-b2bfecf94029
Resource group: myresourcegroup


In [6]:
ws = Workspace.from_config()
experiment_name = 'People_hyperdrive'

experiment=Experiment(ws, experiment_name)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, normal,choice
import os
import shutil
from azureml.core import ScriptRunConfig
from azureml.core import Environment


# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.4)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "learning_rate": normal(10, 3),
    "keep_probability": uniform(0.05, 0.1),
    "--C": uniform(0.02, 1),
    "--max_iter": choice(50, 100, 200),

}) 

if "training" not in os.listdir():
    os.mkdir("training")
    print("Path created")
    shutil.copy("traintest2.py","./training")
    print('copied')

#TODO: Create your estimator and hyperdrive config
compute_target = ws.compute_targets['vacompute']
src = ScriptRunConfig(source_directory='.',
                      script='traintest2.py',
                      arguments=["learning_rate", "keep_probability","--C","--max_iter"],
                      compute_target=compute_target,
                      environment = myenv)

hyperdrive_run_config = HyperDriveConfig(estimator=src,
                             hyperparameter_sampling=param_sampling,
                             policy=early_termination_policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=20,
                             max_concurrent_runs=4)

In [8]:
from azureml.core import Experiment

run = Experiment(ws,'People_hyperdrive').submit(src)
run.wait_for_completion(show_output=True)

RunId: People_hyperdrive_1611416610_bd20057a
Web View: https://ml.azure.com/experiments/People_hyperdrive/runs/People_hyperdrive_1611416610_bd20057a?wsid=/subscriptions/784d8740-4c70-4d20-8f45-b2bfecf94029/resourcegroups/myresourcegroup/workspaces/toria_workspace

Streaming azureml-logs/20_image_build_log.txt

2021/01/23 15:43:41 Downloading source code...
2021/01/23 15:43:43 Finished downloading source code
2021/01/23 15:43:43 Creating Docker network: acb_default_network, driver: 'bridge'
2021/01/23 15:43:44 Successfully set up Docker network: acb_default_network
2021/01/23 15:43:44 Setting up Docker configuration...
2021/01/23 15:43:44 Successfully set up Docker configuration
2021/01/23 15:43:44 Logging in to registry: 55d467844d954e48a295f8417ee4bb2e.azurecr.io
2021/01/23 15:43:46 Successfully logged into 55d467844d954e48a295f8417ee4bb2e.azurecr.io
2021/01/23 15:43:46 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2021/01/23 

In [10]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [11]:
best_model = run.get_metrics()
best_model

{'Regularization Strength:': 1.0,
 'Max iterations:': 1000,
 'Accuracy': 0.9633333333333334}

In [12]:
#TODO: Save the best model
import joblib
joblib.dump(best_model, 'best_remoteml_model.pkl')

['best_remoteml_model.pkl']